## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-31-13-00-00 +0000,missionlocal,Roof! roof!,https://missionlocal.org/2025/08/sf-photos-roo...
1,2025-08-31-12-59-28 +0000,nyt,Judge Temporarily Blocks U.S. Efforts to Depor...,https://www.nytimes.com/2025/08/31/us/politics...
2,2025-08-31-12-58-03 +0000,nyt,"In Trump’s Federal Work Force Cuts, Black Wome...",https://www.nytimes.com/2025/08/31/us/politics...
3,2025-08-31-12-21-59 +0000,nyt,Indonesia’s Leader Says He Will Bow to Some Pr...,https://www.nytimes.com/2025/08/31/world/asia/...
4,2025-08-31-12-14-58 +0000,nypost,Heart-stopping video shows small plane crash i...,https://nypost.com/2025/08/31/us-news/pilot-re...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,26
160,china,10
80,israeli,9
125,new,8
427,killed,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
119,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...,61
62,2025-08-31-05-02-48 +0000,bbc,"Reeling from Trump's tariffs, India and China ...",https://www.bbc.com/news/articles/c0mlen3grx7o...,49
114,2025-08-30-19-37-30 +0000,bbc,Trump administration moves to fire most Voice ...,https://www.bbc.com/news/articles/cdd38v3zvjdo...,49
89,2025-08-31-00-05-56 +0000,nypost,Chicago mayor orders police not to cooperate w...,https://nypost.com/2025/08/30/us-news/chicago-...,47
161,2025-08-30-13-24-47 +0000,nyt,Trump Officials Move to Fire Most Voice of Ame...,https://www.nytimes.com/2025/08/29/us/politics...,47


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
119,61,2025-08-30-18-46-00 +0000,wsj,The Trump administration is drawing up plans t...,https://www.wsj.com/politics/national-security...
51,42,2025-08-31-09-00-00 +0000,wsj,Democratic Party moderates see Virginia govern...,https://www.wsj.com/politics/elections/in-virg...
117,39,2025-08-30-19-00-34 +0000,nypost,Graphic video shows Israeli father throw himse...,https://nypost.com/2025/08/30/world-news/video...
104,35,2025-08-30-21-58-05 +0000,nypost,Tens of thousands of Palestinians flee amid Ga...,https://nypost.com/2025/08/30/world-news/tens-...
31,29,2025-08-31-09-39-53 +0000,bbc,"Putin, Modi and other world leaders in China f...",https://www.bbc.com/news/videos/c5y310qngqpo?a...
21,26,2025-08-31-10-20-35 +0000,nypost,Montana bar shooting suspect Michael Paul Brow...,https://nypost.com/2025/08/31/us-news/montana-...
108,24,2025-08-30-20-39-24 +0000,bbc,Houthis confirm their prime minister killed in...,https://www.bbc.com/news/articles/c620ykrxedwo...
127,24,2025-08-30-17-25-31 +0000,nypost,Russia launches sweeping attack on Southern Uk...,https://nypost.com/2025/08/30/world-news/russi...
6,22,2025-08-31-12-00-00 +0000,startribune,Fans line up for the one place in Minnesota wh...,https://www.startribune.com/minnesota-state-fa...
161,21,2025-08-30-13-24-47 +0000,nyt,Trump Officials Move to Fire Most Voice of Ame...,https://www.nytimes.com/2025/08/29/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
